<a href="https://colab.research.google.com/github/AllanKamimura/AI/blob/master/image/classification/pokemon/final_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

repo: https://github.com/ieee-saocarlos/pokIEEEdex

# Init

## Import libraries and dataset

In [ ]:
import os
import random
import datetime
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from google.colab import files
from google_drive_downloader import GoogleDriveDownloader
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# download image files
# from https://www.kaggle.com/thedagger/pokemon-generation-one

file_id = "1lKfmEjyovEvvZ2zBUkdJwl2fnF4C5OyM"
folder = "dataset"
GoogleDriveDownloader.download_file_from_google_drive(
    file_id = file_id,
    dest_path = "/content/pokemon.zip",
    unzip = True
)

In [ ]:
# dataset class distribution
pokemon_count = {}
index_weight = {}
pokemon_list = os.listdir("/content/{}".format(folder))
pokemon_list.sort()

for index, pokemon in enumerate(pokemon_list):
    pokemon_count[pokemon] = len(os.listdir("/content/{}/{}".format(folder, pokemon)))
    index_weight[index] = 1 / len(os.listdir("/content/{}/{}".format(folder, pokemon)))

index_weight = {k: v * total for total in (sum(index_weight.values()),) for k, v in index_weight.items()}

fig = plt.figure(figsize = (25, 5))
sns.lineplot(x = list(pokemon_count.keys()), y = list(pokemon_count.values())).set_title('Number of images for each pokemon')
plt.xticks(rotation = 90)
plt.margins(x=0)
plt.show()

In [ ]:
seed = 2
height, width = 224, 224

datagen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.1)

train_data = datagen.flow_from_directory(
    directory = "/content/{}".format(folder),
    target_size = (height, width),
    shuffle = True,
    seed = seed,
    batch_size = 15,
    subset = "training"
)

val_data = datagen.flow_from_directory(
    directory = "/content/{}".format(folder),
    target_size = (height, width),
    shuffle = True,
    seed = seed,
    batch_size = 15,
    subset = "validation"
)

indices_class = {value: key for key, value in val_data.class_indices.items()}
n_class = len(train_data.class_indices.keys())
print(indices_class)

# Model 

## model define

In [ ]:
def base_model(shape):
    input = tf.keras.layers.Input(shape = shape, name = "input_layer")
    
    mobile_model = tf.keras.applications.MobileNetV3Small(include_top = False,
                                                          input_shape = shape,
                                                          pooling = "max",
                                                          weights = "imagenet",
                                                          dropout_rate = 0.1,
                                                          )
    
    y = mobile_model(input)

    model = tf.keras.models.Model(
        inputs = input,
        outputs = y,
        name = "mobile_max")  
    return model                 

def my_model(n_class):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 1024, activation = "relu", kernel_initializer = 'he_uniform'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(units = 512, activation = "relu", kernel_initializer = 'he_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(units = 256, activation = "relu", kernel_initializer = 'he_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(units = n_class, activation = "softmax")
    ], name = "my_dense")
    return model

n_class = len(train_data.class_indices.keys())

conv_model = base_model(shape = (height,width,3))
fc_model = my_model(n_class = n_class)

conv_model.trainable = False
model = tf.keras.models.Sequential([conv_model, fc_model], name = "my_model")

model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(
        tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = 0.004, decay_steps = 4000, decay_rate = 0.9
    )),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"]
)

## model train

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)

history = model.fit(train_data,
                    validation_data = val_data,
                    epochs = 100,
                    callbacks = [callback],
                    class_weight = index_weight
                    )   

## model fine-tune

In [ ]:
for layer in model.layers:
    layer.trainable = True

model.summary()

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(
        tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate = 0.0004, decay_steps = 1000, decay_rate = 0.9
    )),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)

history = model.fit(train_data,
                    validation_data = val_data,
                    epochs = 100,
                    callbacks = [callback],
                    class_weight = index_weight
                    )   

In [ ]:
name = "my_save.h5"
model.save("./" + name)

## Model evaluate

In [ ]:
images, labels = val_data.next()
predicts = model.predict(images)

n_rows = (len(images) // 4) + 1
plt.figure(figsize = (12, n_rows * 4))
# plt.subplots_adjust(hspace = 0.1)

for index, image in enumerate(images):
    plt.subplot(n_rows, 4, index + 1)
    image = np.asarray(image, dtype = np.uint8)
    plt.imshow(image)
    plt.title("real: {}\n predict:{}\n score:{:.3f}".format(
        indices_class[np.argmax(labels[index])],
        indices_class[np.argmax(predicts[index])],
        np.max(predicts[index])
    ))